In [2]:
# Import libraries
import sys
import pdal
from osgeo import gdal
import numpy as np
import math
import laspy
import rasterio
import scipy
from scipy import interpolate
from scipy.spatial import ConvexHull, cKDTree, Delaunay
from scipy.ndimage import median_filter, minimum_filter
from matplotlib import cm
from matplotlib import pyplot as plt
from descartes import PolygonPatch
from fiona import collection
from shapely.geometry import shape
import subprocess
from subprocess import run
import json
import zipfile
import os
import time
import threading



# Check versions
print("Python version:", sys.version)
print("Pdal version:", pdal.__version__)
print("Laspy version:", laspy.__version__)
print("numpy version:", np.__version__)
print("------------------------------------")

Python version: 3.9.18 | packaged by conda-forge | (main, Dec 23 2023, 16:35:41) 
[Clang 16.0.6 ]
Pdal version: 3.4.5
Laspy version: 2.5.3
numpy version: 1.26.4
------------------------------------


In [26]:
def chunk_las(las_input: str):
    
    with laspy.open(las_input) as las:

        min_x, max_x = las.header.min[0], las.header.max[0]
        min_y, max_y = las.header.min[1], las.header.max[1]
        min_z, max_z = las.header.min[2], las.header.max[2]
        extent = (min_x, max_x, min_y, max_y, min_z, max_z)

        # Get LAS version
        minor_version = las.header.version.minor
        point_format_id = las.header.point_format.id
        
        # Get XYZ scale and offset
        scale = las.header.scale
        offset = las.header.offset
    

        total_points = las.header.point_count
        num_chunks = (total_points // 30_000_000) + 1
        chunk_index = 0

        for points in las.chunk_iterator(30_000_000):
            # Define output filename for each chunk
            base_name, format = os.path.splitext(las_input)
            output_file = f"{base_name}_{chunk_index}_chunk{format}"

            # Create a new LAS file for this chunk
            with laspy.open(output_file, mode="w", header=las.header) as output_las:
                output_las.write_points(points)
                print(f"Chunk {chunk_index + 1}/{num_chunks} saved as {output_file}")
                
            chunk_index += 1

    return {
            'extent': extent,
            'minor_version': minor_version,
            'point_format_id': point_format_id,
            'scale': scale,
            'offset': offset,
            'num_chunks':num_chunks,
        }
    
start_time = time.time()
results = chunk_las('/Users/yitingcheng/Documents/projection/Daniel Herstine Scan Data.las')
end_time = time.time()
print(f"Chunk finished -- processing time: {end_time - start_time} seconds")

Chunk 1/20 saved as /Users/yitingcheng/Documents/projection/Daniel Herstine Scan Data_0_chunk.las
Chunk 2/20 saved as /Users/yitingcheng/Documents/projection/Daniel Herstine Scan Data_1_chunk.las
Chunk 3/20 saved as /Users/yitingcheng/Documents/projection/Daniel Herstine Scan Data_2_chunk.las
Chunk 4/20 saved as /Users/yitingcheng/Documents/projection/Daniel Herstine Scan Data_3_chunk.las
Chunk 5/20 saved as /Users/yitingcheng/Documents/projection/Daniel Herstine Scan Data_4_chunk.las
Chunk 6/20 saved as /Users/yitingcheng/Documents/projection/Daniel Herstine Scan Data_5_chunk.las
Chunk 7/20 saved as /Users/yitingcheng/Documents/projection/Daniel Herstine Scan Data_6_chunk.las
Chunk 8/20 saved as /Users/yitingcheng/Documents/projection/Daniel Herstine Scan Data_7_chunk.las
Chunk 9/20 saved as /Users/yitingcheng/Documents/projection/Daniel Herstine Scan Data_8_chunk.las
Chunk 10/20 saved as /Users/yitingcheng/Documents/projection/Daniel Herstine Scan Data_9_chunk.las
Chunk 11/20 saved a

In [24]:
def convert_copc():
    
    check_pipeline = [
        "/Users/yitingcheng/Documents/projection/Daniel Herstine Scan Data_19_chunk.las",
        { "type":"writers.copc", "filename":"Daniel Herstine Scan Data_19_chunk.copc.laz"}
    ]
    pdal_command = ["pdal", "pipeline", "--stdin", "-v", "8"]
    run(pdal_command, input=json.dumps(check_pipeline).encode(), check=True)

start_time = time.time()
convert_copc()
end_time = time.time()
print(f"COPC conversion finished -- processing time: {end_time - start_time} seconds")

(PDAL Debug) Debugging...
(pdal pipeline Debug) Executing pipeline in standard mode.


COPC conversion finished -- processing time: 30.788501977920532 seconds


In [3]:
def merge_copc():
    
    check_pipeline = [
        "Daniel Herstine Scan Data_0_chunk.copc.laz",
        "Daniel Herstine Scan Data_1_chunk.copc.laz",
        "Daniel Herstine Scan Data_2_chunk.copc.laz",
        "Daniel Herstine Scan Data_3_chunk.copc.laz",
        "Daniel Herstine Scan Data_4_chunk.copc.laz",
        "Daniel Herstine Scan Data_5_chunk.copc.laz",
        "Daniel Herstine Scan Data_6_chunk.copc.laz",
        "Daniel Herstine Scan Data_7_chunk.copc.laz",
        "Daniel Herstine Scan Data_8_chunk.copc.laz",
        "Daniel Herstine Scan Data_9_chunk.copc.laz",
        "Daniel Herstine Scan Data_10_chunk.copc.laz",
        "Daniel Herstine Scan Data_11_chunk.copc.laz",
        "Daniel Herstine Scan Data_12_chunk.copc.laz",
        "Daniel Herstine Scan Data_13_chunk.copc.laz",
        "Daniel Herstine Scan Data_14_chunk.copc.laz",
        "Daniel Herstine Scan Data_15_chunk.copc.laz",
        "Daniel Herstine Scan Data_16_chunk.copc.laz",
        "Daniel Herstine Scan Data_17_chunk.copc.laz",
        "Daniel Herstine Scan Data_18_chunk.copc.laz",
        "Daniel Herstine Scan Data_19_chunk.copc.laz",
        {"type" : "filters.merge"},
        "Daniel Herstine Scan Data_merge.copc.laz"
    ]
    pdal_command = ["pdal", "pipeline", "--stdin", "-v", "8"]
    run(pdal_command, input=json.dumps(check_pipeline).encode(), check=True)

start_time = time.time()
merge_copc()
end_time = time.time()
print(f"COPC merge finished -- processing time: {end_time - start_time} seconds")

(PDAL Debug) Debugging...
(pdal pipeline Debug) Executing pipeline in standard mode.
(pdal pipeline readers.copc Debug) 1458 overlapping nodes
(pdal pipeline readers.copc Debug) 1960 overlapping nodes
(pdal pipeline readers.copc Debug) 1074 overlapping nodes
(pdal pipeline readers.copc Debug) 1086 overlapping nodes
(pdal pipeline readers.copc Debug) 1512 overlapping nodes
(pdal pipeline readers.copc Debug) 1768 overlapping nodes
(pdal pipeline readers.copc Debug) 1668 overlapping nodes
(pdal pipeline readers.copc Debug) 1335 overlapping nodes
(pdal pipeline readers.copc Debug) 1229 overlapping nodes
(pdal pipeline readers.copc Debug) 1380 overlapping nodes
(pdal pipeline readers.copc Debug) 1323 overlapping nodes
(pdal pipeline readers.copc Debug) 1189 overlapping nodes
(pdal pipeline readers.copc Debug) 1174 overlapping nodes
(pdal pipeline readers.copc Debug) 1140 overlapping nodes
(pdal pipeline readers.copc Debug) 1366 overlapping nodes
(pdal pipeline readers.copc Debug) 2411 overl

COPC merge finished -- processing time: 1662.5285313129425 seconds
